# Imports

In [1]:
# TO DO
# Finish commenting functions
# Add titles and labels to graphs
# Add Min/Max scaling
# scrape news/headlines, build regressions off of data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib.pylab import rcParams
import numpy as np
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose

from statsmodels.tsa.arima_model import ARMA
from sklearn.metrics import mean_squared_error
from pmdarima import auto_arima

from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf

from fbprophet import Prophet
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation
from fbprophet.plot import plot_cross_validation_metric

import warnings
warnings.filterwarnings("ignore")


# Pre Process Function

In [3]:
# Prepares the data for modeling
def preprocess(init_data, exog=True, facebook=False, logged=False):
    
    # Log transform data if wanted
    if logged:
        data = init_data.copy()
        for i in range(1,len(init_data.columns)):
            col = init_data.columns[i]
            data[col] = np.log(init_data[col])
    else:
        data = init_data.copy()
    
    ## Drops unwanted columns and returns dataset
    
    # Preprocess specific to facebook prophet
    if facebook:
        fb = data.copy()
        fb['Date'] = pd.DatetimeIndex(fb['Date'])
        fb = fb.drop(['Open','High','Low','Close','Volume'],axis=1)
        fb = fb.rename(columns={'Date': 'ds','Adj Close':'y'})
        return fb
    
    # If using volume, splits data into 2 separate series for modeling
    elif exog:
        X = data.drop(['Open','High','Low','Close'],axis=1)
        X['Date'] = pd.to_datetime(X['Date'])
        X = X.set_index('Date')
#         start = X['Date'][0]
#         end = X['Date'][len(X)-1]
        return X['Adj Close'],X['Volume']
    else:
        X = data.drop(['Open','High','Low','Close','Volume'],axis=1)
        X['Date'] = pd.to_datetime(X['Date'])
        X = X.set_index('Date')
        return X


In [4]:
def roi_calc(start,end):
    return round((end-start)/start*100,2)

In [5]:
# Returns amount of periods to difference data, using adfuller method
def return_d(data,alpha=0.05, plotting = False, output = False):
    # Uses differencing and adfuller method to find d value for ARIMA models
    diff = data.copy()
    d = 0
    
    # Iterates through a default range of 30 periods and finds the first period where differenced data is stationary
    for j in range(30):
        dtest = adfuller(diff)
        if dtest[1] < alpha:
            
            # Plots differenced data
            if plotting:
                diff.plot()
                plt.xlabel('Date')
                plt.ylabel('Price')
                plt.title(('Differencing with Periods ='+str(j)))
            
            # Prints stat values
            if output:
                dfoutput = pd.Series(dtest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
                for key,value in dtest[4].items():
                    dfoutput['Critical Value (%s)'%key] = value
                print(dfoutput)
            d = j
            return d
        else:
            # Differences the data one additional period and checks adfuller method
            diff = diff.diff(periods=j+1).dropna()
    return d              
        

In [6]:
# Returns suggestion for moving average's p
def return_p(data,alpha=0.05, plotting = False):
    # Determine if data needs differencing first
    d = return_d(data,alpha)
    new_data = data.copy()
    if d > 0:
        new_data = data.diff(periods=d).dropna()
    
    # Uses pacf to find p value for ARIMA models
    data_pacf = pacf(new_data)
    p = 0

    # Plots PACF
    if plotting:
        plot_pacf(new_data,alpha=alpha)
        plt.xlabel('Lags')
        plt.ylabel('PACF')
    
    # Returns first p value less than alpha
    for k in range(len(data_pacf)):
        if (abs(data_pacf[k])) < alpha:
            p = k-1
            return p
        
    return p

In [7]:
# Returns suggestion for auto regressive's q
def return_q(data, alpha=0.05, plotting = False):
    # Determine if data needs differencing first
    d = return_d(data,alpha)
    new_data = data.copy()
    if d > 0:
        new_data = data.diff(periods=d).dropna()
        
    # uses acf to find q value for ARIMA models
    data_acf = acf(new_data)
    q = 0
    
    # Plots ACF
    if plotting:
        plot_acf(new_data,alpha=alpha)
        plt.xlabel('Lags')
        plt.ylabel('ACF')
    
    # Returns first q value less than alpha
    for i in range(len(data_acf)):
        if (abs(data_acf[i])) < alpha:
            q = i-1
            return q
    return q

In [8]:
# Plots seasonal trends if any
def seasonal(data):
    decomposition = seasonal_decompose(data)

    # Gather the trend, seasonality, and residuals 
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    # Plot gathered statistics
    plt.figure(figsize=(12,8))
    plt.subplot(411)
    plt.plot(data, label='Original', color='blue')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend, label='Trend', color='blue')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality', color='blue')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual, label='Residuals', color='blue')
    plt.legend(loc='best')
    plt.tight_layout()

In [9]:
# Helper function to return order values for use in base model
def model_params(data, exog=True, logged=False):
    if exog:
        new_data, ex = preprocess(init_data=data,exog=exog,logged=logged)
    else:
        new_data = preprocess(init_data=data,exog=exog,logged=logged)
    p = return_p(new_data)
    q = return_q(new_data)
    d = return_d(new_data)
    return p,d,q
    print("Returns: p, d, q")


# Train Test Split Function

In [10]:
# Splits train/test data specific to time series 
def train_test(data,exog=True, percent =.75,facebook=False, logged=False, full=False):
    if full:
        exog=False
        length = len(data)+1
    else:
        length = int(len(data)*percent)
    if exog:
        X,Xv= preprocess(init_data=data,exog=exog,facebook=facebook,logged=logged)

        train, trainv = X.iloc[:length],Xv.iloc[:length]
        test, testv = X.iloc[length:],Xv.iloc[length:]
        return train,trainv,test,testv
    else:
        X = preprocess(init_data=data, exog=exog,facebook=facebook, logged=logged)

        if full:
            future_index = pd.date_range(start=X.index[-1], periods=60,freq='D')
            if facebook:
                future = pd.DataFrame(data=future_index,columns=['ds'])
                train = X.iloc[:length]
                test = future.copy()
            else:
                future = pd.DataFrame(data=future_index,columns=['Date'])
                train = X.iloc[:length]
                test = future.set_index('Date')
        else:
            train = X.iloc[:length]
            test = X.iloc[length:]
        return train, test
        

# Base Model Function

In [11]:
def base_model(data,exog=True, plotting=False, summary=False, mse=False, logged=False, full=False, roi=False):

    # Failsafe just in case
    if full:
        exog=False
        mse=False
    else:
        roi=False
    
    
    
    # Get initial p,d,q from helper function
    p,d,q = model_params(data=data, exog=exog, logged=logged)
    
    # Containers for train and test splits
    trainpreds = pd.DataFrame()
    testpreds = pd.DataFrame()
    
    # Splits the data, depending on modeling with/without exogenous, and models using SARIMAX model
    if exog:
        train,trainv,test,testv = train_test(data=data,exog=exog,logged=logged,full=full)
        sarima = sm.tsa.SARIMAX(train,order=(p,d,q),trend='c',exog=trainv).fit()
        trainpreds = sarima.predict()
        forecast = sarima.get_forecast(len(test), index=test.index, exog=testv)
        testpreds = forecast.predicted_mean
        conf = forecast.conf_int(alpha=.05)
    else:
        train,test= train_test(data=data,exog=exog,logged=logged,full=full)
        sarima = sm.tsa.SARIMAX(train,order=(p,d,q),trend='c').fit()      
        trainpreds = sarima.predict()
        forecast = sarima.get_forecast(len(test), index=test.index)
        testpreds = forecast.predicted_mean
        conf = forecast.conf_int(alpha=.05)
        
    
    # Reverse transforms the data if log transformed initially
    if logged:
        itrain = np.exp(train)
        itest = np.exp(test)
        itrainpreds = np.exp(trainpreds)
        itestpreds = np.exp(testpreds)
        iconf = np.exp(conf)
    else:
        itrain = train.copy()
        itest = test.copy()
        itrainpreds = trainpreds.copy()
        itestpreds = testpreds.copy()
        iconf = conf.copy()
        
    # Plots the data and the forecasts
    if plotting:
        figure = plt.figure(figsize=(15,15))

        plt.plot(itrain.append(itest), label='Original')
        plt.plot(itrainpreds.append(itestpreds),label='Model')

        conf_df = iconf.copy()
        conf_df.columns =  ['y1','y2']
        conf_df['X']=test.index
        plt.fill_between(conf_df['X'],conf_df['y1'],conf_df['y2'], alpha=.2)        
        
        
        
        plt.legend()
        plt.show()
        
        # Plots residual data
        sarima.plot_diagnostics()
    if summary:
        print(sarima.summary())
    if mse:
#         print('Train RMSE: ', mean_squared_error(itrain, itrainpreds)**0.5)
        print('Test RMSE: ', mean_squared_error(itest, itestpreds)**0.5)
#         if aic:
#             print('AIC: ', sarima.aic)

    if roi:
        start = itrainpreds[-1]
        end = itestpreds[-1]
        print("ROI: ", roi_calc(start=start,end=end),"%")
    
    return sarima 

# Auto Arima Function

In [12]:
def create_auto_arima(data, exog=True, plotting=False, summary=False, mse=False, logged=False, full=False, roi = False):
#     p,d,q = model_params(data,exog)
    trainpreds = pd.DataFrame()
    testpreds = pd.DataFrame()
    
    # Failsafe just in case
    if full:
        exog=False
        mse=False
    else:
        roi = False
    
    if exog:
        train,trainv,test,testv = train_test(data=data,exog=exog, logged=logged, full=full)
        auto = auto_arima(y=train ,trace=True, exog=trainv,stepwise=True, max_order=12).fit(train)
#         trainpreds = auto.predict()
        testpreds, conf = auto.predict(len(test), index=test.index, exog=testv, return_conf_int=True)

    else:
        train,test= train_test(data=data,exog=exog, logged=logged, full=full)
        auto = auto_arima(y=train ,trace=True,stepwise=True, max_order=12).fit(train)      
        trainpreds = auto.predict()
        testpreds, conf = auto.predict(len(test), index=test.index, return_conf_int=True)

        
        
    if logged:
        itrain = np.exp(train)
        itest = np.exp(test)
#         itrainpreds = np.exp(trainpreds)
        itestpreds = np.exp(testpreds)
        iconf = np.exp(conf)

    else:
        itrain = train.copy()
        itest = test.copy()
#         itrainpreds = trainpreds.copy()
        itestpreds = testpreds.copy()
#         print(testpreds)
#         print(itestpreds)
        iconf = conf.copy()
        
        
    
    if plotting:
        plot_preds = pd.DataFrame(data=itestpreds, columns=['Adj Close'], index=itest.index)
#         plot_preds['Date'] = test.index
        
        figure = plt.figure(figsize=(10,10))
        
        conf_df = pd.DataFrame(data=iconf, columns = ['y1','y2'])
        conf_df['X']=test.index
        plt.fill_between(conf_df['X'],conf_df['y1'],conf_df['y2'], alpha=.2)
        
        
        plt.plot(itrain.append(itest), label='Original')
        plt.plot(plot_preds,label='Model')
        
        
        
        
        plt.legend()
        plt.show()
        
        auto.plot_diagnostics()
    if summary:
        print(auto.summary())
    if mse:
#         print('Train RMSE: ', mean_squared_error(train, trainpreds)**0.5)
        print('Test RMSE: ', mean_squared_error(itest, itestpreds)**0.5)
#         print("AIC:",auto.aic())

    if roi:
        start = itestpreds[0]
        end = itestpreds[-1]
        print("ROI: ", roi_calc(start=start,end=end),"%")
        
    return auto

# Prophet Function

In [13]:
def create_prophet(data,exog=False,plotting=False,summary=False, mse=False, logged=False, full=False, roi=False):
    fb, fbtest = train_test(data,exog=exog,facebook=True, logged=logged, full=full)
    fb_model = Prophet(interval_width=.95, daily_seasonality=True)
    fb_model.fit(fb)
#     train = fb_model.predict(fb['ds'])
    if full:
        length = pd.date_range(start=fb.iloc[-1].ds, periods=60,freq='D')
        mse = False
    else:
        length = pd.date_range(start=fb.iloc[0].ds, end=fbtest.iloc[-1].ds,freq='D')
        roi = False
    ifuture = pd.DataFrame(data=length,columns=['ds'])
    iforecast = fb_model.predict(ifuture)

    forecast = iforecast.copy()
    
#     if logged:
#         forecast = iforecast.copy()
#         for i in range(1,len(forecast.columns)):
#             col = forecast.columns[i]
#             forecast[col] = np.exp(iforecast[col])
    
    
    if plotting:
        fb_model.plot(forecast,uncertainty=True, figsize=(10,10));
        plt.plot(fb.append(fbtest).set_index('ds'), c='red', label = 'Actual', alpha=.2)
        plt.legend()

    if mse:
        train_error = pd.concat([forecast.set_index('ds'),fb.set_index('ds')], join='inner', axis=1)
        test_error = pd.concat([forecast.set_index('ds'),fbtest.set_index('ds')], join='inner', axis=1)
        
#         print("Train RMSE:", mean_squared_error(train_error.y,train_error.yhat)**.5)
        print("Test RMSE:", mean_squared_error(test_error.y,test_error.yhat)**.5)
#         if aic:
#             train_aic = len(train_error)*np.log(mean_squared_error(train_error.y,train_error.yhat)+2*1)
#             print("AIC:",train_aic)

    if roi:
        start = fb['y'].iloc[-1]
        end = forecast['yhat'].iloc[-1]
        print("ROI: ", roi_calc(start=start,end=end),"%")    

    return fb_model

In [14]:
df = pd.read_csv('Data/MongoDB.csv')

In [15]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,10/19/2017,33.000000,34.000000,29.100000,32.07,32.07,11508500
1,10/20/2017,33.369999,33.369999,30.100000,30.68,30.68,2358700
2,10/23/2017,30.510000,31.330000,30.190001,30.50,30.50,749400
3,10/24/2017,30.459999,30.920000,30.438999,30.57,30.57,420700
4,10/25/2017,30.500000,31.100000,29.879999,31.00,31.00,1219400
